In [5]:
import pandas as pd


train_data = pd.read_csv("data\main\df_train_b1.csv", index_col=0)
test_data = pd.read_csv('data\main\df_test_b1.csv', index_col=0)

In [6]:
if True:

    import pandas as pd
    temp_df = pd.read_csv("./data/detail/df_loan1_b1.csv")
    temp_df = temp_df.dropna(axis=0, subset=["date_1"])

    def chuli_loan(df, colname, ways):
        cust_id = list(set(df['cust_id']))
        res_df = pd.DataFrame({'cust_id': cust_id})
        try:
            temp_train = pd.read_csv('./data/df_train.csv')
            temp_test = pd.read_csv('./data/df_test.csv')
            temp = list(temp_test.columns)
        except:
            temp = []

        ways_1 = []

        global train_data, test_data

        for i in ways:
            if (colname+'_'+i) not in temp:
                ways_1.append(i)
                res_df[colname+'_'+i] = None
            else:
                print(colname+'_'+i+'已存在')
                train_data = pd.merge(train_data, temp_train[['cust_id', colname+'_'+i]], on="cust_id",
                                      how="left", suffixes=("", "_y"))
                test_data = pd.merge(test_data, temp_test[['cust_id', colname+'_'+i]], on="cust_id",
                                     how="left", suffixes=("", "_y"))
        ways = ways_1
        if len(ways) == 0:
            return res_df
        if 'mean' in ways:
            res_df[colname+'_mean'] = df.groupby('cust_id')[colname].mean()
        if 'sum' in ways:
            res_df[colname+'_sum'] = df.groupby('cust_id')[colname].sum()
        if 'count' in ways:
            res_df[colname+'_count'] = df.groupby('cust_id')[colname].count()
        if 'std' in ways:
            res_df[colname+'_std'] = df.groupby('cust_id')[colname].std()
        if 'diff_std' in ways:
            res_df[colname+'_diff_std'] = df.groupby('cust_id')[colname].apply(
                lambda x: x.sort_values(ascending=True).diff()[2:].std())
        return res_df

    for colname in ['loan1_2', 'loan1_3', 'loan1_7', 'loan1_9']:
        df = chuli_loan(temp_df, colname, ('mean', 'sum', 'count', 'std'))
        train_data = pd.merge(train_data, df, on="cust_id",
                              how="left", suffixes=("", "_y"))
        test_data = pd.merge(test_data, df, on="cust_id",
                             how="left", suffixes=("", "_y"))

loan1_2_mean已存在
loan1_2_sum已存在
loan1_2_count已存在
loan1_2_std已存在
loan1_3_mean已存在
loan1_3_sum已存在
loan1_3_count已存在
loan1_3_std已存在
loan1_7_mean已存在
loan1_7_sum已存在
loan1_7_count已存在
loan1_7_std已存在
loan1_9_mean已存在
loan1_9_sum已存在
loan1_9_count已存在
loan1_9_std已存在


In [7]:
df_train=train_data
df_test=test_data

In [8]:
df_train.drop(["loan1_17",'loan1_31'],axis=1,inplace=True)
df_test.drop(["loan1_17",'loan1_31'],axis=1,inplace=True)

In [9]:
def delnan(df):
    df = df.dropna(how="all", axis=1)  # 删除全是空值的列
    df = df.fillna(-99)
    return df

df_train = delnan(train_data)
df_test = delnan(test_data)

In [10]:
full_col_name = list(df_train.columns)
basic_col = [i for i in full_col_name if "basic" in i]
loan_col = [i for i in full_col_name if "loan" in i]
overdue_col = [i for i in full_col_name if "overdue" in i]
query_col = [i for i in full_col_name if "query" in i]
judicial_col = [i for i in full_col_name if "judicial" in i]
date_col = [i for i in full_col_name if "date" in i]

In [11]:
x_col = basic_col+['is_judicial']+loan_col+query_col+overdue_col+['scope_num'] + \
    ['industry_num']+['date_3_count', 'date_1_std', 'date_1_diff_std']
x_col=[i for i in x_col if i in full_col_name]

In [12]:
print(x_col)

['basic_1', 'basic_2', 'basic_3', 'basic_4', 'basic_5', 'basic_6', 'basic_7', 'basic_8', 'basic_9', 'basic_10', 'basic_11', 'basic_12', 'basic_13', 'basic_14', 'basic_15', 'loan1_1', 'loan1_2', 'loan1_3', 'loan1_4', 'loan1_5', 'loan1_6', 'loan1_7', 'loan1_8', 'loan1_9', 'loan1_10', 'loan1_11', 'loan1_12', 'loan1_13', 'loan1_14', 'loan1_15', 'loan1_16', 'loan1_18', 'loan1_19', 'loan1_20', 'loan1_21', 'loan1_22', 'loan1_23', 'loan1_24', 'loan1_25', 'loan1_26', 'loan1_27', 'loan1_28', 'loan1_29', 'loan1_30', 'loan1_32', 'loan1_33', 'loan2_1', 'loan2_2', 'loan2_3', 'loan2_4', 'loan2_5', 'loan2_6', 'loan2_7', 'loan2_8', 'loan2_9', 'loan2_10', 'loan2_11', 'loan2_12', 'loan1_2_mean', 'loan1_2_sum', 'loan1_2_count', 'loan1_2_std', 'loan1_3_mean', 'loan1_3_sum', 'loan1_3_count', 'loan1_3_std', 'loan1_7_mean', 'loan1_7_sum', 'loan1_7_count', 'loan1_7_std', 'loan1_9_mean', 'loan1_9_sum', 'loan1_9_count', 'loan1_9_std', 'query_1', 'query_2', 'query_3', 'query_4', 'query_5', 'query_6', 'query_7', '

In [13]:
import copy
import xgboost as xgb
import random
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation


In [89]:
X = df_train[x_col]
Y = df_train['label']

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, Y, test_size=3000, stratify=Y, random_state=2022)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
x_perd = xgb.DMatrix(df_test[x_col])

evallist = [(dtrain,"train"),(dtest, 'eval')]
param = {
    'max_depth': 5,
    'eta': 0.008,
    'colsample_bytree': 0.5,
    'objective': 'binary:logistic',
    # 'verbosity':0,
    'scale_pos_weight': 1,
    'eval_metric': 'auc',
    'subsample': 0.55,
    'min_child_weight': 10,
    'gamma': 0.5,
    'seed': 666,
    'tree_method': 'gpu_hist',
    'gpu_id': 0,
}

bst = xgb.train(param, dtrain, 3000, evals=evallist,early_stopping_rounds=300)

[0]	train-auc:0.58742	eval-auc:0.52430
[1]	train-auc:0.58600	eval-auc:0.53211
[2]	train-auc:0.58799	eval-auc:0.52422
[3]	train-auc:0.58799	eval-auc:0.52422
[4]	train-auc:0.65184	eval-auc:0.57864
[5]	train-auc:0.65869	eval-auc:0.57045
[6]	train-auc:0.66150	eval-auc:0.57311
[7]	train-auc:0.66150	eval-auc:0.57311
[8]	train-auc:0.66156	eval-auc:0.57489
[9]	train-auc:0.66277	eval-auc:0.57089
[10]	train-auc:0.66277	eval-auc:0.57089
[11]	train-auc:0.66336	eval-auc:0.57080
[12]	train-auc:0.66473	eval-auc:0.56942
[13]	train-auc:0.67034	eval-auc:0.57299
[14]	train-auc:0.67783	eval-auc:0.57898
[15]	train-auc:0.68055	eval-auc:0.58417
[16]	train-auc:0.68044	eval-auc:0.58577
[17]	train-auc:0.68063	eval-auc:0.59110
[18]	train-auc:0.68041	eval-auc:0.58970
[19]	train-auc:0.68001	eval-auc:0.59014
[20]	train-auc:0.67938	eval-auc:0.58982
[21]	train-auc:0.68065	eval-auc:0.58977
[22]	train-auc:0.67907	eval-auc:0.58693
[23]	train-auc:0.67818	eval-auc:0.58628
[24]	train-auc:0.68018	eval-auc:0.58661
[25]	train

In [60]:
bst.eval(dtest)

'[0]\teval-auc:0.72389342210314078'

In [61]:
xgb_pred_label = list(bst.predict(x_perd))

In [101]:
lgbcallback = [early_stopping(300), log_evaluation(10)]
param = {
        'task': 'train',
        "max_depth": 5,
        'boosting_type': 'gbdt',  # 设置提升类型
        'objective': 'binary',  # 目标函数
        'metric': 'auc',  # 评估函数
        'learning_rate': 0.01,  # 学习速率
        # 建树的特征选择比例
        'feature_fraction': 1,
        'bagging_fraction': 0.7,  # 建树的样本采样比例
        'bagging_freq': 1,  # k 意味着每 k 次迭代执行bagging
        'min_data_in_leaf': 1,
        'verbose': -1,  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
        'device_type': 'gpu',
        'seed':666,
    }

dtrain = lgb.Dataset(X_train, y_train)
dtest = lgb.Dataset(X_test, y_test)
bst1 = lgb.train(param, dtrain, 3000, valid_sets=[
            dtest], callbacks=lgbcallback)

Training until validation scores don't improve for 300 rounds
[10]	valid_0's auc: 0.624922
[20]	valid_0's auc: 0.634269
[30]	valid_0's auc: 0.647964
[40]	valid_0's auc: 0.648456
[50]	valid_0's auc: 0.652368
[60]	valid_0's auc: 0.66179
[70]	valid_0's auc: 0.662233
[80]	valid_0's auc: 0.666675
[90]	valid_0's auc: 0.674265
[100]	valid_0's auc: 0.672414
[110]	valid_0's auc: 0.678328
[120]	valid_0's auc: 0.685364
[130]	valid_0's auc: 0.684641
[140]	valid_0's auc: 0.688098
[150]	valid_0's auc: 0.691322
[160]	valid_0's auc: 0.691988
[170]	valid_0's auc: 0.691437
[180]	valid_0's auc: 0.694877
[190]	valid_0's auc: 0.697107
[200]	valid_0's auc: 0.695957
[210]	valid_0's auc: 0.696958
[220]	valid_0's auc: 0.699242
[230]	valid_0's auc: 0.699245
[240]	valid_0's auc: 0.700341
[250]	valid_0's auc: 0.700892
[260]	valid_0's auc: 0.70206
[270]	valid_0's auc: 0.702851
[280]	valid_0's auc: 0.704539
[290]	valid_0's auc: 0.70596
[300]	valid_0's auc: 0.708071
[310]	valid_0's auc: 0.709756
[320]	valid_0's auc: